In [129]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

xl = pd.ExcelFile("Data Ta Idol musim 1-12 fixed.xlsx")
xl.sheet_names

['Sheet1']

In [130]:
df = xl.parse("Sheet1")
df.head()

,No.,Nama Peserta,Lagu & Penyanyi Asli,Hasil,Genre,Babak,Musim
0,1,Helena Andrian,"Yang Ku 'Tunggu"" - Anggun",Lolos,Pop,Grup 1,1
1,2,Raymond Latuheru,"""Firasat"" - Marcell Siahaan",Eliminasi,Pop,Grup 1,1
2,3,Tiara Degrasia,"""Kissing A Fool"" - George Michael",Eliminasi,Funk,Grup 1,1
3,4,Aldy Erlangga,"""One Last Cry"" - Brian McKnight",Eliminasi,R&B,Grup 1,1
4,5,Imelda Ragillia,"""Cinta Kita"" - Reza Artamevia",Eliminasi,Pop,Grup 1,1


In [207]:
label_encoder = LabelEncoder()
df['genre_encoded'] = label_encoder.fit_transform(df['Genre'])
target_values = ['Eliminasi', 'Tidak Lolos']

df['hasil'] = [0 if x in target_values else 1 for x in df['Hasil']]

df.head()

,No.,Nama Peserta,Lagu & Penyanyi Asli,Hasil,Genre,Babak,Musim,genre_encoded,hasil
0,1,Helena Andrian,"Yang Ku 'Tunggu"" - Anggun",Lolos,Pop,Grup 1,1,27,1
1,2,Raymond Latuheru,"""Firasat"" - Marcell Siahaan",Eliminasi,Pop,Grup 1,1,27,0
2,3,Tiara Degrasia,"""Kissing A Fool"" - George Michael",Eliminasi,Funk,Grup 1,1,16,0
3,4,Aldy Erlangga,"""One Last Cry"" - Brian McKnight",Eliminasi,R&B,Grup 1,1,32,0
4,5,Imelda Ragillia,"""Cinta Kita"" - Reza Artamevia",Eliminasi,Pop,Grup 1,1,27,0


In [219]:
## Pisahkan fitur dan target
X = df[['genre_encoded']]
y = df['hasil']

# Bagi data menjadi training dan testing set 70 % untuk training dan 30% untuk testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define sigmoid function
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

# Create a simple neural network with a sigmoid activation function
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x
# Adjust output_size based on the number of possible 'hasil' values (0,1) meaka output_size = 2, sedangkan input_size tergantung terhadap parameter inputan
# dalam hal ini masih menggunakan 1 field [gendre] yang memiliki jumlah 1
model = NeuralNetwork(input_size=1, output_size=2)  

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Train the model num_epoch  bertujuan untuk melakukan iterasi terhadap model sehingga mendapatkan accuration yang optimal
num_epochs = 400
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


# Test the model
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    print(f'Accuracy: {torch.sum(predicted == y_test).item() / float(y_test.size(0)) * 100}%')

Accuracy: 88.59649122807018%
